In [171]:
import sys

In [172]:
sys.path

['/srv/share3/hagrawal9/project/habitat/habitat-api',
 '/srv/share3/hagrawal9/project/habitat/habitat-sim',
 '/coc/pskynet3/hagrawal9/project/habitat/habitat-api/notebooks',
 '/srv/share3/hagrawal9/envs/habitat/lib/python36.zip',
 '/srv/share3/hagrawal9/envs/habitat/lib/python3.6',
 '/srv/share3/hagrawal9/envs/habitat/lib/python3.6/lib-dynload',
 '',
 '/srv/share3/hagrawal9/envs/habitat/lib/python3.6/site-packages',
 '/coc/pskynet3/hagrawal9/project/habitat/habitat-api',
 '/srv/share3/hagrawal9/envs/habitat/lib/python3.6/site-packages/tensorflow-1.13.1-py3.6-linux-x86_64.egg',
 '/srv/share3/hagrawal9/envs/habitat/lib/python3.6/site-packages/tb_nightly-2.4.0a20200812-py3.6.egg',
 '/srv/share3/hagrawal9/envs/habitat/lib/python3.6/site-packages/moviepy-2.0.0.dev1-py3.6.egg',
 '/srv/share3/hagrawal9/envs/habitat/lib/python3.6/site-packages/wheel-0.34.2-py3.6.egg',
 '/srv/share3/hagrawal9/envs/habitat/lib/python3.6/site-packages/tensorflow_estimator-1.13.0-py3.6.egg',
 '/srv/share3/hagrawal

In [173]:
cd '/srv/share3/hagrawal9/project/habitat/habitat-api/'

/coc/pskynet3/hagrawal9/project/habitat/habitat-api


In [71]:
import gzip
import habitat
import habitat_sim
import habitat_sim.bindings as hsim
import json
import matplotlib.pyplot as plt
import magnum as mn
import numpy as np
import os
import pandas as pd
import tqdm 

from habitat.config import Config
from habitat_sim.physics import MotionType
from habitat_sim.attributes import ObjectAttributes
from pathlib import Path
from PIL import Image
from typing import Any, Dict, List, Optional
from habitat.utils.visualizations.maps import *

%matplotlib inline

In [72]:
# %load_ext autoreload
# %autoreload 2
# from habitat.tasks.sokoban.sokoban_vis_utils import SokobanVisualization

In [73]:
def make_cfg(settings):
    sim_cfg = habitat_sim.SimulatorConfiguration()
    sim_cfg.gpu_device_id = 0
    sim_cfg.default_agent_id = settings["default_agent_id"]
    sim_cfg.scene.id = settings["scene"]
    sim_cfg.enable_physics = settings["enable_physics"]
    sim_cfg.physics_config_file = settings["physics_config_file"]

    # Note: all sensors must have the same resolution
    sensors = {
        "rgb": {
            "sensor_type": habitat_sim.SensorType.COLOR,
            "resolution": [settings["height"], settings["width"]],
            "position": [0.0, settings["sensor_height"], 0.0],
            "hfov": settings["hfov"]
        },
        "depth": {
            "sensor_type": habitat_sim.SensorType.DEPTH,
            "resolution": [settings["height"], settings["width"]],
            "position": [0.0, settings["sensor_height"], 0.0],
            "hfov": settings["hfov"]
        },
    }

    sensor_specs = []
    for sensor_uuid, sensor_params in sensors.items():
        if settings[sensor_uuid]:
            sensor_spec = habitat_sim.SensorSpec()
            sensor_spec.uuid = sensor_uuid
            sensor_spec.sensor_type = sensor_params["sensor_type"]
            sensor_spec.resolution = sensor_params["resolution"]
            sensor_spec.position = sensor_params["position"]
            sensor_spec.parameters["hfov"] = str(sensor_params["hfov"])

            sensor_specs.append(sensor_spec)

    # Here you can specify the amount of displacement in a forward action and the turn angle
    agent_cfg = habitat_sim.agent.AgentConfiguration()
    agent_cfg.sensor_specifications = sensor_specs
    agent_cfg.action_space = {
        "move_forward": habitat_sim.agent.ActionSpec(
            "move_forward", habitat_sim.agent.ActuationSpec(amount=0.1)
        ),
        "turn_left": habitat_sim.agent.ActionSpec(
            "turn_left", habitat_sim.agent.ActuationSpec(amount=10.0)
        ),
        "turn_right": habitat_sim.agent.ActionSpec(
            "turn_right", habitat_sim.agent.ActuationSpec(amount=10.0)
        ),
    }

    return habitat_sim.Configuration(sim_cfg, [agent_cfg])

In [74]:
settings = {
    "max_frames": 10,
    "width": 640,  # Spatial resolution of the observations
    "height": 480,
    "hfov": 79,
    "scene": "data/scene_datasets/gibson_train_val/Barboursville.glb",  # Scene path
    "default_agent_id": 0,
    "sensor_height": 0.88,  # Height of sensors in meters
    "rgb": True,  # RGB sensor
    "depth": True,  # Depth sensor
    "seed": 1,
    "enable_physics": True,
    "physics_config_file": "data/default.phys_scene_config.json", 
    "silent": False, 
    "num_objects": 10,
    "compute_shortest_path": False,
    "compute_action_shortest_path": False,
    "save_png": True
}

In [75]:
navmesh_settings = habitat_sim.NavMeshSettings()
navmesh_settings.set_defaults()
navmesh_settings.agent_radius = 0.2
navmesh_settings.agent_height = 0.88
navmesh_settings.agent_max_climb = 0.01

In [76]:
def register_object_templates(sim, object_templates):
    obj_attr_mgr = sim.get_object_template_manager()
    handles = obj_attr_mgr.get_file_template_handles()

    for sel_file_obj_handle, template_info in object_templates.items():
        obj_handle = os.path.basename(sel_file_obj_handle).split('.')[0]
        object_handle = obj_attr_mgr.get_file_template_handles(obj_handle)[0]

        obj_template = obj_attr_mgr.get_template_by_handle(object_handle)
        obj_template.scale = np.array(template_info['scale'])
        obj_attr_mgr.register_template(obj_template)

In [77]:
def init_agent(sim):
    obj_attr_mgr = sim.get_object_template_manager()
    handles = obj_attr_mgr.get_file_template_handles()
    object_handle = obj_attr_mgr.get_file_template_handles('sphere')[0]
    # Place the agent
    agent_pos = sim.pathfinder.get_random_navigable_point()
    # print(agent_pos)
    
    sim.agents[0].scene_node.translation = agent_pos
    
    agent_orientation_y = np.random.randint(0, 360)
    sim.agents[0].scene_node.rotation = mn.Quaternion.rotation(
        mn.Deg(agent_orientation_y), mn.Vector3(0, 1.0, 0)
    )
    
    agent_object_id = sim.add_object_by_handle(object_handle)
    sim.set_translation(agent_pos, agent_object_id)
    
    return sim.get_agent(0).get_state(), agent_object_id

In [78]:
def get_rotation(sim, oid):
    quat = sim.get_rotation(oid)
    return np.array(quat.vector).tolist() + [quat.scalar]

In [102]:
def geodesic_distance(sim, position_a, position_b):
    path = habitat_sim.MultiGoalShortestPath()
    path.requested_start = np.array(position_a, dtype=np.float32)
    if isinstance(position_b[0], List) or isinstance(
        position_b[0], np.ndarray
    ):
        path.requested_ends = np.array(position_b, dtype=np.float32)
    else:
        path.requested_ends = np.array(
            [np.array(position_b, dtype=np.float32)]
        )

    sim.pathfinder.find_path(path)

    return path.geodesic_distance

def euclidean_distance(position_a, position_b):
    return np.linalg.norm(
        np.array(position_b) - np.array(position_a), ord=2
    )

def validate_object(sim, agent_position, object_position, goal_position, object_positions, goal_positions, dist_threshold=15.0):
    agent_position = sim.agents[0].scene_node.translation
    ao_geo_dist = geodesic_distance(sim, agent_position, [object_position])
    ag_geo_dist = geodesic_distance(sim, agent_position, [goal_position])
    og_geo_dist = geodesic_distance(sim, object_position, [goal_position])
    
    ao_l2_dist = euclidean_distance(agent_position, object_position)
    ag_l2_dist = euclidean_distance(agent_position, goal_position)
    og_l2_dist = euclidean_distance(object_position, goal_position)
    
    ao_dist_ratio = ao_geo_dist / ao_l2_dist
    og_dist_ratio = og_geo_dist / og_l2_dist
    
    if ao_l2_dist < 1.0 or ao_geo_dist > 100 or np.abs(object_position[1] - agent_position[1]) > 0.2:
        # import pdb
        # pdb.set_trace()
        # print("agent object distance:", ao_l2_dist, ao_geo_dist)
        return False
    
    if ag_geo_dist > 100 or np.abs(goal_position[1] - agent_position[1]) > 0.2:
        # print("agent goal distance failing:", ag_geo_dist)
        return False
    
    if og_l2_dist < dist_threshold or og_geo_dist > 100 or np.abs(object_position[1] - goal_position[1]) > 0.2:
        # print("object goal distance! :", og_l2_dist, og_geo_dist)
        return False
    
    for j, curr_pos in enumerate([object_position, goal_position]):
        for i, pos in enumerate(object_positions + goal_positions):
            geo_dist = geodesic_distance(sim, curr_pos, [pos])
            l2_dist = euclidean_distance(curr_pos, pos)
            
            # check height difference to assure s and are from same floor
            if np.abs(curr_pos[1] - pos[1]) > 0.2: 
                # print("different height!")
                return False
            
            if sim.pathfinder.island_radius(curr_pos) != sim.pathfinder.island_radius(pos):
                # print("different island!")
                return False 
            
            if l2_dist < 1.0 or geo_dist > 100:
                # print("distance between two objects less!")
                return False

    return True
    
def validate_again(sim, object_positions, goal_positions):
    agent_position = sim.agents[0].scene_node.translation
    
    for i, posi in enumerate([agent_position] + object_positions + goal_positions):
        for j, posj in enumerate(object_positions + goal_positions):
            if np.abs(posi[1] - posj[1]) > 0.2:  # check height difference to assure s and t are from the same floor
                print("diff floor", i, j, posi, posj)
                return False

            geo_dist = geodesic_distance(sim, posi, [posj])

            if geo_dist > 100:
                print(geo_dist, i, j)
                return False

    return True

In [103]:
def init_episode_dict(sim, scene, episode_num):
    episode_dict = {
        'episode_id': episode_num, 
        'scene_id': scene,
        'start_position': np.array(sim.get_translation(0)).tolist(), 
        'start_rotation': get_rotation(sim, 0), 
        'info': {}, 
        'objects': [

        ],
        'goals': [ 
            
        ], 
    }
    return episode_dict

def add_object_details(sim, episode_dict, num_objects, object_idxs, object_template_idxs):
    for i in range(num_objects):
        obj_id = object_idxs[i]
        object_template = {
            'object_id': i, 
            'object_handle': object_template_idxs[i],
            'position': np.array(sim.get_translation(obj_id)).tolist(), 
            'rotation': get_rotation(sim, obj_id),
        }
        episode_dict['objects'].append(object_template)
    
    return episode_dict

def add_goal_details(sim, episode_dict, num_objects, goal_idxs):
    for i in range(num_objects):
        goal_id = goal_idxs[i]
        goal_template = {
            'position': np.array(sim.get_translation(goal_id)).tolist(), 
            'rotation': get_rotation(sim, goal_id),
        }
            
        episode_dict['goals'].append(goal_template)
    return episode_dict

In [104]:
def set_object_on_top_of_surface(sim, obj_id):
    r"""
    Adds an object in front of the agent at some distance.
    """

    obj_node = sim.get_object_scene_node(obj_id)
    xform_bb = habitat_sim.geo.get_transformed_bb(
        obj_node.cumulative_bb, obj_node.transformation
    )

    # also account for collision margin of the scene
    scene_collision_margin = 0.00
    y_translation = mn.Vector3(
        0, xform_bb.size_y() / 2.0 + scene_collision_margin, 0
    )
    sim.set_translation(y_translation + sim.get_translation(obj_id), obj_id)
    
    return np.array(sim.get_translation(obj_id))

In [123]:
def init_test_scene_new(sim, object_templates, navmesh_settings, num_objects, dist_thresh=5.0): 
    object_positions = []
    goal_positions = []
    object_idxs = []
    goal_idxs = []
    object_template_idxs = []
    
    obj_attr_mgr = sim.get_object_template_manager()
    agent_position = sim.agents[0].scene_node.translation
    
    
    for obj_id in range(num_objects):
        count = 0
        
        object_template_id = np.random.choice(list(object_templates.keys()))
        object_handle = obj_attr_mgr.get_file_template_handles(object_template_id)[0]
        rotation_x = mn.Quaternion.rotation(mn.Deg(-90), mn.Vector3(1.0, 0, 0))
        rotation_y = mn.Quaternion.rotation(mn.Deg(90), mn.Vector3(0.0, 1.0, 0))
        rotation_z = mn.Quaternion.rotation(mn.Deg(0), mn.Vector3(0.0, 0, 1.0))
        # rotation_x1 = mn.Quaternion.rotation(mn.Deg(-45), mn.Vector3(1.0, 0, 0))
        orientation = rotation_z * rotation_y * rotation_x

        object_id = sim.add_object_by_handle(object_handle)
        goal_id = sim.add_object_by_handle(object_handle)

        
        while count < 100:
            i = 0
            while(True):
                object_position = sim.pathfinder.get_random_navigable_point()
                # object_position[1] = agent_position[1]
                # set_object_on_top_of_surface(sim, object_id)
                
                object_dist = sim.pathfinder.distance_to_closest_obstacle(object_position, max_search_radius=2.0)
                ao_geo_dist = geodesic_distance(sim, agent_position, [object_position])
                
                if object_dist > 0.5 and ao_geo_dist < 100:
                    break
                # else:
                #     print("\r close to an obstacle! {} {}".format(object_dist, i), end = " ")
                i+=1
            i = 0
            while(True):
                goal_position = sim.pathfinder.get_random_navigable_point()
                # goal_position[1] = agent_position[1]
                # set_object_on_top_of_surface(sim, goal_id)
                
                goal_dist = sim.pathfinder.distance_to_closest_obstacle(goal_position, max_search_radius=2.0)
                ao_geo_dist = geodesic_distance(sim, agent_position, [goal_position])
                if goal_dist > 0.5 and ao_geo_dist < 100:
                    break
                # else:
                    # print("\r close to an obstacle! {} {}".format(object_dist, i), end = " ")
                i+= 1
            
            sim.set_object_motion_type(MotionType.DYNAMIC, object_id)
            sim.set_object_motion_type(MotionType.DYNAMIC, goal_id)
            
            
            # print("\n Object Dist: {}; Goal Dist: {}".format(object_dist, goal_dist))
            sim.set_translation(object_position, object_id)
            sim.set_translation(goal_position, goal_id)
            sim.set_rotation(orientation, object_id)
            sim.set_rotation(orientation, goal_id)
            # print(object_position, sim.get_translation(object_id))
            object_position = set_object_on_top_of_surface(sim, object_id)
            goal_position = set_object_on_top_of_surface(sim, goal_id)
            # print(object_position, sim.get_translation(object_id))
            
            sim.set_object_motion_type(MotionType.STATIC, object_id)
            sim.set_object_motion_type(MotionType.STATIC, goal_id)
            
            sim.recompute_navmesh(
                sim.pathfinder, 
                navmesh_settings,
                include_static_objects=True
            )
            
#             if sim.pathfinder.is_navigable(object_position):
#                 print("init obj loc navigable")
#                 continue
#             if sim.pathfinder.is_navigable(goal_position):
#                 print("final obj loc navigable")
#                 continue
#             else:
#                 print("not navigable")
#             lower_bound, upper_bound = sim.pathfinder.get_bounds()
#             meters_per_pixel = min(
#                 abs(upper_bound[coord] - lower_bound[coord]) / 1024
#                 for coord in [0, 2]
#             )
#             print(lower_bound, upper_bound, meters_per_pixel)
#             top_down_map = sim.pathfinder.get_topdown_view(
#                 meters_per_pixel=meters_per_pixel, height=sim.get_agent(0).state.position[1]
#             ).astype(np.uint8)
#             plt.imshow(top_down_map * 255)
#             plt.show()

            if validate_object(
                sim, agent_position, object_position, goal_position, 
                object_positions, goal_positions, 
                dist_threshold=dist_thresh
            ) and validate_again(sim, object_positions, goal_positions):
#                 print(
#                     "added object: "
#                     + str(object_id)
#                     + " at: "
#                     + str(object_position)
#                 )
                break
            
            # plt.close()
            count += 1
        
        if count < 100:
            object_positions.append(object_position)
            goal_positions.append(goal_position)    
            object_idxs.append(object_id)
            goal_idxs.append(goal_id)
            object_template_idxs.append(object_template_id)
            # print("Success in {}".format(count))
        else:
            sim.remove_object(object_id)
            sim.remove_object(goal_id)
            # print("couldn't find a good layout")
            return object_positions, goal_positions, object_idxs, goal_idxs, object_template_idxs

        # recompute navmesh so that objects don't overlap with other existing objects. 
        sim.recompute_navmesh(
            sim.pathfinder, 
            navmesh_settings,
            include_static_objects=True
        )
        
    return object_positions, goal_positions, object_idxs, goal_idxs, object_template_idxs
        

In [124]:
# set the number of objects to 1 always for now. 
def build_episode(episode_num, object_templates, num_objects=5):
    episodes = {'episodes': []}
    # for scene_id in train_df['id'].tolist()[5:10]:
    for scene_id in train_df[train_df['id'] == 'Cokeville']['id'].tolist():
        scene = 'data/scene_datasets/gibson_train_val/{}.glb'.format(scene_id)
        settings['scene'] = scene
        print(scene)
        
        cfg = make_cfg(settings)
        with habitat_sim.Simulator(cfg) as sim:
             # clear the objects if we are re-running this initializer
            for old_obj_id in sim.get_existing_object_ids():
                # print(old_obj_id)
                sim.remove_object(old_obj_id)

            sim.recompute_navmesh(sim.pathfinder, navmesh_settings, True)
            # sv = SokobanVisualization(sim, navmesh_settings, map_resolution=(250, 250), num_samples=20000, draw_border=True)

            episode = 0
            dist_thresh = 3.0

            while episode < episode_num:
                start_state, agent_object_id = init_agent(sim)
                
                if sim.pathfinder.island_radius(start_state.position) < 3:
                    # print("small radius. continuing", start_state.position)
                    continue 
                    
                sim.recompute_navmesh(sim.pathfinder, navmesh_settings, True)

                num_object = np.random.choice(range(2, num_objects + 1))
                num_object = 5
                object_positions, goal_positions, object_idxs, goal_idxs, object_template_idxs = init_test_scene_new(
                    sim, object_templates, navmesh_settings, num_object, dist_thresh
                )

                sim.recompute_navmesh(sim.pathfinder, navmesh_settings, False)
                result = validate_again(sim, object_positions, goal_positions)
                if result == False or len(object_idxs) == 0:
                    continue 

                num_object = len(object_idxs)

                assert len(object_idxs) == len(goal_idxs)
                
                episode_dict = init_episode_dict(sim, scene, episode)
                episode_dict = add_object_details(sim, episode_dict, num_object, object_idxs, object_template_idxs)
                episode_dict = add_goal_details(sim, episode_dict, num_object, goal_idxs)
                episodes['episodes'].append(episode_dict)
                print("\r Episode {} Object {}".format(episode, num_object), end=" ")
                episode += 1

            print("")    
    return episodes

In [370]:
train_df = pd.read_pickle('data/sokoban_gibson_test.pkl')

In [278]:
train_df = train_df.append({'id':'Islandton'}, ignore_index=True)

In [281]:
train_df

,id,name,split_full,split_full+,split_medium,split_tiny,area,floor,navigation_complexity,room,ssa
457,Scandinavia,model-535,train,train,none,none,531.017,3.0,2.885,22.0,0.794
184,Fredericksburg,model-401,train,train,none,none,149.214,1.0,1.511,2.0,0.872
222,Hainesburg,model-511,train,train,train,none,275.235,3.0,3.244,15.0,0.873
365,Monson,model-135,train,train,none,none,188.722,3.0,3.146,8.0,0.909
551,Westerville,model-297,train,train,none,none,359.709,3.0,2.337,13.0,0.944
554,Whitethorn,model-154,train,train,none,none,108.069,1.0,1.210,6.0,1.008
441,Roeville,model-304,train,train,none,none,484.984,4.0,1.030,21.0,1.008
524,Trail,model-162,train,train,none,none,535.171,4.0,3.123,20.0,1.046
246,Hobson,model-208,train,train,none,none,125.348,1.0,3.187,9.0,1.069
492,Stilwell,model-514,train,train,train,none,358.399,3.0,7.782,16.0,1.136


In [236]:
# train_df.to_pickle('data/sokoban_gibson_train.pkl')

In [126]:
object_templates = {}
with open('data/ycb_object_templates.json') as f:
    object_templates = json.load(f)

In [ ]:
episodes = build_episode(10, object_templates, num_objects=5)

In [ ]:
episodes['object_templates'] = object_templates

In [ ]:
episodes['episodes']

In [327]:
def scene_statistics(split):
    scene_list = []
    for filename in os.listdir('data/datasets/rearrangement/gibson/v1/{}/content/'.format(split)):
        if 'v1' in filename:
            scene_id = filename.split('.')[0].split('_')[-1]
            filesize = os.path.getsize('data/scene_datasets/gibson_train_val/'+scene_id+'.glb')
            with gzip.open('data/datasets/rearrangement/gibson/v1/{}/content/'.format(split) + filename, "rt") as f:
                scene_data = json.load(f)
            object_count = []
            for episode in scene_data['episodes']:
                object_count.append(len(episode['objects']))
            
            scene_list.append({
                'scene_id': scene_id,
                'file_size':filesize, 
                'file_name': filename,
                'object_count': np.mean(object_count),
                'episode_length': len(scene_data['episodes'])
            })
        
            
    scene_df = pd.DataFrame.from_dict(scene_list)
    scene_df['file_size'] = scene_df['file_size'].apply(lambda x: x/(1024 * 1024))
    # print(scene_df.sort_values(by='file_size', ascending=False))
    return scene_df

In [331]:
train_scenes_df = scene_statistics('train')

In [332]:
val_scenes_df = scene_statistics('val')

In [333]:
test_scenes_df = scene_statistics('test')

In [336]:
train_scenes_df.sort_values(by='file_size', ascending=False)

,scene_id,file_size,file_name,object_count,episode_length
29,Grantsville,35.989517,rearrangement_v1_train_n=1000_o=5_Grantsville....,3.186,1000
6,Ewell,35.859791,rearrangement_v1_train_n=1000_o=5_Ewell.json.gz,3.266,1000
31,Trail,30.732159,rearrangement_v1_train_n=1000_o=5_Trail.json.gz,3.465,1000
32,Barranquitas,29.165432,rearrangement_v1_train_n=1000_o=5_Barranquitas...,3.440,1000
1,Scandinavia,27.427521,rearrangement_v1_train_n=1000_o=5_Scandinavia....,3.554,1000
13,Westerville,26.226624,rearrangement_v1_train_n=1000_o=5_Westerville....,3.130,1000
28,Stanleyville,25.759068,rearrangement_v1_train_n=1000_o=5_Stanleyville...,3.485,1000
11,Hainesburg,24.833599,rearrangement_v1_train_n=1000_o=5_Hainesburg.j...,3.482,1000
22,Stilwell,24.396706,rearrangement_v1_train_n=1000_o=5_Stilwell.jso...,3.443,1000
30,Ovalo,24.092197,rearrangement_v1_train_n=1000_o=5_Ovalo.json.gz,2.979,1000


In [337]:
train_df.shape

(35, 11)

In [340]:
val_scenes_df

,scene_id,file_size,file_name,object_count,episode_length
0,Chrisney,15.651398,rearrangement_v1_val_n=100_o=-1_Chrisney.json.gz,5.0,100
1,Sasakwa,22.133743,rearrangement_v1_val_n=100_o=-1_Sasakwa.json.gz,5.0,100
2,Kemblesville,17.457623,rearrangement_v1_val_n=100_o=-1_Kemblesville.j...,5.0,100
3,Sodaville,11.920403,rearrangement_v1_val_n=100_o=-1_Sodaville.json.gz,5.0,100
4,Willow,13.391956,rearrangement_v1_val_n=100_o=-1_Willow.json.gz,5.0,100
5,Model,21.220291,rearrangement_v1_val_n=100_o=-1_Model.json.gz,5.0,100
6,Eagerville,13.564564,rearrangement_v1_val_n=100_o=-1_Eagerville.jso...,5.0,100
7,Castor,24.503319,rearrangement_v1_val_n=100_o=-1_Castor.json.gz,5.0,100
8,Hallettsville,25.917988,rearrangement_v1_val_n=100_o=-1_Hallettsville....,5.0,100
9,Maugansville,38.809502,rearrangement_v1_val_n=100_o=-1_Maugansville.j...,5.0,100


In [341]:
from collections import defaultdict

In [374]:
def scene_stats(split, max_count=1000):
    scene_list = defaultdict(list)
    for filename in os.listdir('data/datasets/rearrangement/gibson/v1/{}/content/'.format(split)):
        if 'v1' in filename:
            scene_id = filename.split('.')[0].split('_')[-1]
            filesize = os.path.getsize('data/scene_datasets/gibson_train_val/'+scene_id+'.glb')
            with gzip.open('data/datasets/rearrangement/gibson/v1/{}/content/'.format(split) + filename, "rt") as f:
                scene_data = json.load(f)
            
            for episode in scene_data['episodes']:
                if episode['scene_id']!='data/scene_datasets/gibson_train_val/'+scene_id+'.glb':
                    print(filename)
                    break
                    
#                 scene_list[episode['scene_id']].append(1)
        
    return scene_list

In [379]:
scene_list = scene_stats('train')

rearrangement_v1_train_n=1000_o=5_Stanleyville.json.gz
rearrangement_v1_train_n=1000_o=5_Barranquitas.json.gz
